In [1]:
import pyspark 
from pyspark.sql import SparkSession
import base64

In [2]:
from pyspark.sql.types import StringType

In [3]:
key_file = open("/home/bda_crypto_busters/repos/BigDataAnalytics/2_data_preprocessing/crypto/stream/crypto-busting-375023-6722d6967eca.json", "rb")
key = base64.b64encode(key_file.read())
key = key.decode("utf-8")

In [5]:
spark = SparkSession.builder \
    .master("yarn") \
    .appName("Pub/Sub Lite to Spark Streaming") \
    .config("spark.jars", "/home/bda_crypto_busters/repos/BigDataAnalytics/2_data_preprocessing/crypto/stream/pubsublite-spark-sql-streaming-LATEST-with-dependencies.jar") \
    .config("spark.dynamicAllocation.enabled", "false") \
    .getOrCreate()

### Reading streaming data

In [6]:
project_number = 1072423212419
location = "europe-central2"
subscription_id = "bda-coinbase-sub-lite"
df = (
    spark.readStream.format("pubsublite")
    .option(
        "pubsublite.subscription",
        f"projects/{project_number}/locations/{location}/subscriptions/{subscription_id}",
    )
    .option("gcp.credentials.key", key)
    .load()
)

In [7]:
# w tej kolumnie powinny być dane ale są zakodowane jako json?
df = df.withColumn('data', df.data.cast(StringType())).select('data')

In [7]:
# Print
res = df.writeStream \
    .format("console") \
    .outputMode("append") \
    .start()

23/01/22 13:59:38 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-b9fa6222-d1b2-43e3-b3ac-4d7848753936. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/22 13:59:38 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [8]:
from pyspark.sql.functions import from_json
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, FloatType

In [9]:
JSONschema = StructType([ 
    StructField("side", StringType(), True), 
    StructField("price", StringType(), False), 
    StructField("product_id", StringType(), False),
    StructField("time", StringType(), False), 
])

In [10]:
sdf = df.withColumn("JSONData", from_json(col("data"), JSONschema)).select("JSONData.*")

In [11]:
sdf = sdf.withColumn("price", sdf.price.cast(FloatType()))

In [12]:
sdf.printSchema()

root
 |-- side: string (nullable = true)
 |-- price: float (nullable = true)
 |-- product_id: string (nullable = true)
 |-- time: string (nullable = true)



In [17]:
sdf.isStreaming

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:40527)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 977, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1115, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 111] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:40527)

In [6]:
from google.cloud import bigtable
from google.cloud.bigtable import column_family
from google.auth.credentials import Credentials
from google.oauth2 import service_account
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/home/bda_crypto_busters/repos/BigDataAnalytics/2_data_preprocessing/reddit/bigtable_key.json"
credentials = service_account.Credentials.from_service_account_file("/home/bda_crypto_busters/repos/BigDataAnalytics/2_data_preprocessing/reddit/bigtable_key.json")

client = bigtable.Client(project="crypto-busting-375023", admin=True, credentials=credentials, read_only=False)
instance = client.instance(instance_id="bda-bigtable-instance")
table = instance.table(table_id="coinbase-table")


import datetime

from google.cloud import bigtable


def write_simple(project_id, instance_id, table_id, row):
    client = bigtable.Client(project=project_id, admin=True)
    instance = client.instance(instance_id)
    table = instance.table(table_id)

    timestamp = datetime.datetime.utcnow()
    column_family_id = "coinbase-cf"

    row_key = "phone#4c410523#20190501"

    
    # new row_key - timestamp???
    
    row_key = timestamp
    
    roww = table.direct_row(row_key)
    
    thisRowValues = []
    for r in row:
        thisRowValues.append(r)
    
    
    roww.set_cell(column_family_id, thisRowValues[0], thisRowValues[1], thisRowValues[2],thisRowValues[3])
    roww.commit()

    print("Successfully wrote row {}.".format(row_key))




In [ ]:
def process_row(row):
    # Write row to storage - how to write this????
    write_simple("crypto-busting-375023", "bda-bigtable-instance", "coinbase-table", row)
    pass



In [ ]:
query = sdf.writeStream.foreach(process_row).start() 

In [19]:
# not ok, few lines below

sdf.createOrReplaceTempView("updates")

In [20]:
sdf2 = spark.sql("select count(*) from updates")

In [21]:
sdf2.isStreaming

True

In [ ]:
# getting list of rows using collect()
row_list = sdf.collect()
  
# Printing the first Row object
# from which data is extracted
print(row_list[0]

In [14]:
sdf.toPandas()

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33723)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gatewa

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33723)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gatewa

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33723)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gatewa

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33723)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1207, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1033, in send_command
    response = connection.send_command(command)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1211, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while receiving

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gatewa

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:33723)
Traceback (most recent call last):
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py", line 3433, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_17241/933583626.py", line 1, in <module>
    sdf.toPandas()
  File "/usr/lib/spark/python/pyspark/sql/pandas/conversion.py", line 67, in toPandas
    timezone = self.sql_ctx._conf.sessionLocalTimeZone()
  File "/usr/lib/spark/python/pyspark/sql/context.py", line 108, in _conf
    return self.sparkSession._jsparkSession.sessionState().conf()
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/java_gateway.py", line 1304, in __call__
    return_value = get_return_value(
  File "/usr/lib/spark/python/pyspark/sql/utils.py", line 111, in deco
    return f(*a, **kw)
  File "/opt/conda/miniconda3/lib/python3.8/site-packages/py4j/protocol.py", line 334, in g

Py4JError: An error occurred while calling o63.sessionState

In [12]:
res = sdf.writeStream \
      .format("console") \
      .outputMode("append") \
      .start()

23/01/22 14:41:19 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-96aa6a19-8248-484d-9438-3cc785ca3766. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/22 14:41:19 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


-------------------------------------------
Batch: 0
-------------------------------------------
+----+--------+----------+--------------------+
|side|   price|product_id|                time|
+----+--------+----------+--------------------+
|    |  0.3354|   ADA-USD|2023-01-20T11:54:...|
|    | 1549.45|   ETH-USD|2023-01-20T11:54:...|
|    | 20976.5|   BTC-USD|2023-01-20T11:49:...|
|    |  0.3358|   ADA-USD|2023-01-20T11:49:...|
|    | 0.08105|  DOGE-USD|2023-01-20T11:49:...|
|    |   21.16|   SOL-USD|2023-01-20T11:49:...|
|    | 1550.98|   ETH-USD|2023-01-20T11:49:...|
|    | 0.08099|  DOGE-USD|2023-01-20T11:54:...|
|    |   21.15|   SOL-USD|2023-01-20T11:54:...|
|    |20960.31|   BTC-USD|2023-01-20T11:54:...|
|    |  1549.4|   ETH-USD|2023-01-20T12:04:...|
|    |   21.18|   SOL-USD|2023-01-20T12:09:...|
|    |  0.3358|   ADA-USD|2023-01-20T12:09:...|
|    |20969.36|   BTC-USD|2023-01-20T12:04:...|
|    | 0.08105|  DOGE-USD|2023-01-20T12:09:...|
|    |20979.05|   BTC-USD|2023-01-20T12

In [30]:
res

In [15]:
query = sdf.groupBy("product_id").count()

In [16]:
(
    query.writeStream
    .format("memory")
    .outputMode("complete")
    .queryName("crypto_count")
    .start()
)

23/01/22 14:01:26 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-43dc81ad-2743-456a-aa9f-7df5e1becb82. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/22 14:01:26 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [18]:
sth = spark.sql("select * from crypto_count")

In [19]:
type(sth)

pyspark.sql.dataframe.DataFrame

In [20]:
type(res)

pyspark.sql.streaming.StreamingQuery

In [21]:
type(sdf)

pyspark.sql.dataframe.DataFrame

In [22]:
type(query)

pyspark.sql.dataframe.DataFrame

In [25]:
# sdf to pandas

(
    sdf.writeStream
    .format("memory")
    .outputMode("append")
    .queryName("crypto_view")
    .start()
)
sth = spark.sql("select * from crypto_view")
pandasik = sth.toPandas()



23/01/22 14:08:32 WARN org.apache.spark.sql.streaming.StreamingQueryManager: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-3e636588-ce3c-4b67-bb85-dbfdcfbc3bcd. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/01/22 14:08:32 WARN org.apache.spark.sql.streaming.StreamingQueryManager: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


In [28]:
pandasik


,side,price,product_id,time
